In [13]:
import torch
import numpy as np
import sparseconvnet as scn

In [3]:


# Use the GPU if there is one, otherwise CPU
use_gpu = torch.cuda.is_available()

model = scn.Sequential().add(
    scn.SparseVggNet(2, 1,
                     [['C',  8], ['C',  8], ['MP', 3, 2],
                      ['C', 16], ['C', 16], ['MP', 3, 2],
                      ['C', 24], ['C', 24], ['MP', 3, 2]])
).add(
    scn.SubmanifoldConvolution(2, 24, 32, 3, False)
).add(
    scn.BatchNormReLU(32)
).add(
    scn.SparseToDense(2,32)
)
if use_gpu:
    model.cuda()

# output will be 10x10
inputSpatialSize = model.input_spatial_size(torch.LongTensor([10, 10]))
input = scn.InputBatch(2, inputSpatialSize)

msg = [
    " X   X  XXX  X    X    XX     X       X   XX   XXX   X    XXX   ",
    " X   X  X    X    X   X  X    X       X  X  X  X  X  X    X  X  ",
    " XXXXX  XX   X    X   X  X    X   X   X  X  X  XXX   X    X   X ",
    " X   X  X    X    X   X  X     X X X X   X  X  X  X  X    X  X  ",
    " X   X  XXX  XXX  XXX  XX       X   X     XX   X  X  XXX  XXX   "]

#Add a sample using setLocation
input.addSample()
for y, line in enumerate(msg):
    for x, c in enumerate(line):
        if c == 'X':
            location = torch.LongTensor([x, y])
            featureVector = torch.FloatTensor([1])
            input.setLocation(location, featureVector, 0)

#Add a sample using setLocations
input.addSample()
locations = []
features = []
for y, line in enumerate(msg):
    for x, c in enumerate(line):
        if c == 'X':
            locations.append([x,y])
            features.append([1])
locations = torch.LongTensor(locations)
features = torch.FloatTensor(features)
input.setLocations(locations, features, 0)

# Optional: allow metadata preprocessing to be done in batch preparation threads
# to improve GPU utilization.
#
# Parameter:
#    3 if using MP3/2 pooling or C3/2 convolutions for downsizing,
#    2 if using MP2 pooling for downsizing.
input.precomputeMetadata(3)

model.train()
if use_gpu:
    input.cuda()
output = model.forward(input)

# Output is 2x32x10x10: our minibatch has 2 samples, the network has 32 output
# feature planes, and 10x10 is the spatial size of the output.
print(output.size(), output.data.type())

(torch.Size([2, 32, 10, 10]), 'torch.cuda.FloatTensor')


In [3]:
output

Variable containing:
(0 ,0 ,.,.) = 
 -0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 -0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 -0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
           ...             ⋱             ...          
  1.8311  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000

(0 ,1 ,.,.) = 
 -0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 -0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 -0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
           ...             ⋱             ...          
  0.7370  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000

(0 ,2 ,.,.) = 
 -0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  1.1436  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.2771  0.0000  0.0000  ...   0.0000  0.0000  0.00

In [11]:
model.input_spatial_size(torch.LongTensor([1, 2]))


 15
 23
[torch.LongTensor of size 2]

In [23]:
a = np.ones((2,3,4))

In [24]:
a.shape

(2, 3, 4)

In [27]:
np.stack([a]*5, axis=3).shape

(2, 3, 4, 5)

In [29]:
torch.ones((3,))


 1
 1
 1
[torch.FloatTensor of size 3]

In [31]:
torch.nonzero(torch.zeros((3,)))

[torch.LongTensor with no dimension]

In [36]:
np.dtype(np.arange(5)[0])

dtype('int64')